<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/Housing_predictions_Clayton/Dummies_House_Price_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/test_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("test_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

+----+----------+-----------+-------+-----------+-----------+---------+------------+----------+---------+---------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+---------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+-----------+------------+----------+----------+----------+----------+----------+-----------+-------------+-----------+--------+------+-----+-------+------+------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------

In [ ]:
# Import RandomForestRegressor and Pickle

from sklearn.ensemble import RandomForestRegressor
import pickle

In [ ]:
#Load model
#NOTE: Must manually load file before un-pickling

model = pickle.load(open("VLC_Ames_Housing_Price_Model", 'rb'))

In [ ]:
X_predict = df[["firstflrsf", 
                "GrLivArea", 
                "LotArea", 
                "GarageArea", 
                "BsmtUnfSF", 
                "TotalBsmtSF", 
                "LotFrontage", 
                "GarageYrBlt", 
                "MoSold", 
                "YearBuilt"]]

y_predict = model.predict(X_predict)

print(y_predict)

[132189.99 161919.5  185122.   ... 153574.18 113218.   232581.23]


In [34]:
import pandas as pd

prices = y_predict.tolist()